# **Metabolic cost computations in iML1515**

In [2]:
import cobra
import pandas as pd
from cobra.io import load_model 
from cobra.medium import minimal_medium

In [8]:
# Load the iML1515 model and run statistics

model = cobra.io.read_sbml_model("./data/models/iML1515_new.xml")
model.summary()
print(model.summary())

print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

Objective
1.0 BIOMASS_Ec_iML1515_core_75p37M = 0.8697726420320139

Uptake
------
Metabolite     Reaction      Flux  C-Number  C-Flux
     ca2_e     EX_ca2_e  0.004527         0   0.00%
      cl_e      EX_cl_e  0.004527         0   0.00%
 cobalt2_e EX_cobalt2_e 2.174E-05         0   0.00%
     cu2_e     EX_cu2_e 0.0006167         0   0.00%
     fe2_e     EX_fe2_e   0.01397         0   0.00%
  glc__D_e  EX_glc__D_e        10         6 100.00%
       k_e       EX_k_e    0.1698         0   0.00%
     mg2_e     EX_mg2_e  0.007545         0   0.00%
     mn2_e     EX_mn2_e  0.000601         0   0.00%
    mobd_e    EX_mobd_e 6.088E-06         0   0.00%
     nh4_e     EX_nh4_e     9.393         0   0.00%
     ni2_e     EX_ni2_e 0.0002809         0   0.00%
      o2_e      EX_o2_e     22.44         0   0.00%
      pi_e      EX_pi_e     0.839         0   0.00%
     so4_e     EX_so4_e     0.219         0   0.00%
     zn2_e     EX_zn2_e 0.0002966         0   0.00%

Secretion
---------
Metabolite    

In [9]:
# Fix the glucose uptake rate and set up the medium for metabolite production 

minimal_medium(model, 0.1, minimize_components=True)

glucose_rxn = model.reactions.get_by_id("EX_glc__D_e")
glucose_rxn.lower_bound = -10

model.reactions.get_by_id("EX_o2_e").upper_bound= 1000
model.reactions.get_by_id("EX_nh4_e").upper_bound= 1000
model.reactions.get_by_id("EX_pi_e").upper_bound= 1000

c:\Users\joaqu\AppData\Local\Programs\Python\Python310\lib\site-packages\cobra\medium\minimal_medium.py:112: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  medium = pd.Series()


In [10]:
# List of amino acids
amino_acids = ['ala__L', 'arg__L', 'asn__L', 'asp__L', 'cys__L', 'gln__L', 'glu__L', 
               'gly', 'his__L', 'ile__L', 'leu__L', 'lys__L', 'met__L', 'phe__L',
               'pro__L', 'ser__L', 'thr__L', 'trp__L', 'tyr__L', 'val__L']

# Initialise an empty DataFrame to store the results
amino_acid_flux_df = pd.DataFrame(columns=['Reaction', 'Flux', 'Glucose_Uptake', 'Yield'])

# Optimize the flux of each amino acid exchange reaction
for aa in amino_acids:
    ex_rxn_id = f"EX_{aa}_e"
    ex_rxn = model.reactions.get_by_id(ex_rxn_id)
    
    # Set the objective to the exchange reaction of the amino acid
    model.objective = ex_rxn
    
    # Optimize the model
    solution = model.optimize()
    flux = solution.objective_value
    
    # Calculate flux per glucose uptake
    glc_yield = flux / abs(glucose_rxn.lower_bound)
    
    # Create a new DataFrame with the results for the current amino acid
    aa_flux_df = pd.DataFrame({'Reaction': [ex_rxn_id], 'Flux': [flux], 'Glucose_Uptake': [abs(glucose_rxn.lower_bound)], 'Yield': [round(glc_yield,2)]})

    # Concatenate the new DataFrame to the existing amino_acid_flux_df
    amino_acid_flux_df = pd.concat([amino_acid_flux_df, aa_flux_df], ignore_index=True)

# Display the final DataFrame
print(amino_acid_flux_df)

       Reaction       Flux Glucose_Uptake  Yield
0   EX_ala__L_e  19.859623             10   1.99
1   EX_arg__L_e   8.274286             10   0.83
2   EX_asn__L_e  15.935354             10   1.59
3   EX_asp__L_e  17.975172             10   1.80
4   EX_cys__L_e   9.839501             10   0.98
5   EX_gln__L_e   0.000000             10   0.00
6   EX_glu__L_e  11.814928             10   1.18
7      EX_gly_e  27.012174             10   2.70
8   EX_his__L_e   8.062692             10   0.81
9   EX_ile__L_e   7.240000             10   0.72
10  EX_leu__L_e   7.513617             10   0.75
11  EX_lys__L_e   7.722667             10   0.77
12  EX_met__L_e   5.909447             10   0.59
13  EX_phe__L_e   5.427523             10   0.54
14  EX_pro__L_e  10.019104             10   1.00
15  EX_ser__L_e  20.625882             10   2.06
16  EX_thr__L_e  12.325000             10   1.23
17  EX_trp__L_e   4.464906             10   0.45
18  EX_tyr__L_e   5.635459             10   0.56
19  EX_val__L_e   9.

In [11]:
# Save the DataFrame to a CSV file
amino_acid_flux_df.to_csv('./metabolic_cost/iML1515_amino_acid_flux_file.csv', index=False)